In [ ]:
# %% Cell 1: Setup
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.config import get_config
from src.data.data_loader import DataLoader
from src.features.temporal_features import TemporalFeatures

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Imports successful")

# %% Cell 2: Load Data
config = get_config()
loader = DataLoader(config)

print("Loading machine data...")
df = loader.load_machine_data("machine_001")

sampling_rate = config.get('data_generation.sampling_rate', 100)
print(f"Sampling rate: {sampling_rate} Hz")

df_sample = df.iloc[:50000].copy()
print(f"Data shape: {df_sample.shape}")

# %% Cell 3: Trend Analysis
print("\n" + "="*80)
print("TREND ANALYSIS")
print("="*80)

extractor = TemporalFeatures(sampling_rate=sampling_rate)

# Extract a window
window = df_sample['vibration_rms'].iloc[:1000].values

# Extract trend features
trend_features = extractor.extract_trend_features(window, prefix='vib_')

print("\nTrend Features:")
for key in ['vib_trend_slope', 'vib_trend_r_squared', 'vib_trend_strength',
            'vib_curvature', 'vib_monotonicity_increasing']:
    if key in trend_features:
        print(f"  {key}: {trend_features[key]:.4f}")

# Visualize trend
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(len(window))
ax.plot(x, window, alpha=0.6, label='Signal', linewidth=1)

# Plot linear trend
slope = trend_features['vib_trend_slope']
intercept = trend_features['vib_trend_intercept']
trend_line = slope * x + intercept
ax.plot(x, trend_line, 'r--', linewidth=2, label=f'Linear Trend (slope={slope:.4f})')

# Plot polynomial trend
if 'vib_poly_coef_0' in trend_features:
    poly_coef = [trend_features['vib_poly_coef_0'],
                 trend_features['vib_poly_coef_1'],
                 trend_features['vib_poly_coef_2']]
    poly_trend = np.polyval(poly_coef, x)
    ax.plot(x, poly_trend, 'g--', linewidth=2, label='Polynomial Trend')

ax.set_xlabel('Sample', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.set_title('Trend Analysis', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/trend_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# %% Cell 4: Autocorrelation Analysis
print("\n" + "="*80)
print("AUTOCORRELATION ANALYSIS")
print("="*80)

acf_features = extractor.extract_autocorrelation_features(window, max_lag=100, prefix='vib_')

print("\nAutocorrelation Features:")
for key, value in list(acf_features.items())[:8]:
    print(f"  {key}: {value:.4f}")

# Plot autocorrelation
fig, ax = plt.subplots(figsize=(14, 6))

lags = range(1, 101)
acf_values = []

# Compute ACF
signal_norm = (window - np.mean(window)) / (np.std(window) + 1e-10)
for lag in lags:
    acf = np.corrcoef(signal_norm[:-lag], signal_norm[lag:])[0, 1]
    acf_values.append(acf)

ax.plot(lags, acf_values, linewidth=2)
ax.axhline(0, color='black', linestyle='--', linewidth=1)
ax.axhline(0.05, color='red', linestyle='--', linewidth=1, alpha=0.5, label='95% confidence')
ax.axhline(-0.05, color='red', linestyle='--', linewidth=1, alpha=0.5)

ax.set_xlabel('Lag', fontsize=12)
ax.set_ylabel('Autocorrelation', fontsize=12)
ax.set_title('Autocorrelation Function', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/autocorrelation.png', dpi=300, bbox_inches='tight')
plt.show()

# %% Cell 5: Cross-Correlation Between Sensors
print("\n" + "="*80)
print("CROSS-CORRELATION ANALYSIS")
print("="*80)

# Get signals
vib_signal = df_sample['vibration_rms'].iloc[:1000].values
temp_signal = df_sample['temperature'].iloc[:1000].values

# Extract cross-correlation
xcorr_features = extractor.extract_cross_correlation_features(
    vib_signal, temp_signal, max_lag=50, prefix='vib_temp_'
)

print("\nCross-Correlation Features:")
for key, value in xcorr_features.items():
    print(f"  {key}: {value:.4f}")

# Plot cross-correlation
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Normalized signals
axes[0].plot(vib_signal, label='Vibration RMS', alpha=0.7, linewidth=1)
ax2 = axes[0].twinx()
ax2.plot(temp_signal, label='Temperature', color='orange', alpha=0.7, linewidth=1)
axes[0].set_ylabel('Vibration', fontsize=10)
ax2.set_ylabel('Temperature', fontsize=10)
axes[0].set_title('Signals Over Time', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper left', fontsize=9)
ax2.legend(loc='upper right', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Cross-correlation
max_lag = 50
lags = range(-max_lag, max_lag+1)
xcorr_vals = []

vib_norm = (vib_signal - np.mean(vib_signal)) / (np.std(vib_signal) + 1e-10)
temp_norm = (temp_signal - np.mean(temp_signal)) / (np.std(temp_signal) + 1e-10)

for lag in lags:
    if lag == 0:
        xcorr = np.corrcoef(vib_norm, temp_norm)[0, 1]
    elif lag > 0:
        xcorr = np.corrcoef(vib_norm[lag:], temp_norm[:-lag])[0, 1]
    else:
        xcorr = np.corrcoef(vib_norm[:lag], temp_norm[-lag:])[0, 1]
    xcorr_vals.append(xcorr)

axes[1].plot(lags, xcorr_vals, linewidth=2)
axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
axes[1].axvline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[1].set_xlabel('Lag', fontsize=12)
axes[1].set_ylabel('Cross-Correlation', fontsize=12)
axes[1].set_title('Cross-Correlation: Vibration vs Temperature', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/cross_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

# %% Cell 6: Degradation Features
print("\n" + "="*80)
print("DEGRADATION ANALYSIS")
print("="*80)

# Compare normal vs anomaly windows
normal_window = df_sample[df_sample['is_anomaly'] == 0]['vibration_rms'].iloc[:1000].values
anomaly_window = df_sample[df_sample['is_anomaly'] == 1]['vibration_rms'].iloc[:1000].values

if len(anomaly_window) > 0:
    # Extract degradation features
    deg_normal = extractor.extract_degradation_features(normal_window, prefix='normal_')
    deg_anomaly = extractor.extract_degradation_features(anomaly_window, prefix='anomaly_')
    
    print("\nDegradation Features Comparison:")
    print("\nNormal:")
    for key in ['normal_degradation_trend', 'normal_variance_growth', 'normal_mean_shift']:
        if key in deg_normal:
            print(f"  {key}: {deg_normal[key]:.4f}")
    
    print("\nAnomaly:")
    for key in ['anomaly_degradation_trend', 'anomaly_variance_growth', 'anomaly_mean_shift']:
        if key in deg_anomaly:
            print(f"  {key}: {deg_anomaly[key]:.4f}")
    
    # Visualize
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Normal
    axes[0].plot(normal_window, color='green', alpha=0.7, linewidth=1)
    x = np.arange(len(normal_window))
    trend = deg_normal['normal_degradation_trend'] * x + np.mean(normal_window)
    axes[0].plot(x, trend, 'r--', linewidth=2, label='Trend')
    axes[0].set_ylabel('Amplitude', fontsize=10)
    axes[0].set_title('Normal Operation', fontsize=12, fontweight='bold')
    axes[0].legend(fontsize=9)
    axes[0].grid(True, alpha=0.3)
    
    # Anomaly
    axes[1].plot(anomaly_window, color='red', alpha=0.7, linewidth=1)
    x = np.arange(len(anomaly_window))
    trend = deg_anomaly['anomaly_degradation_trend'] * x + np.mean(anomaly_window)
    axes[1].plot(x, trend, 'black', linestyle='--', linewidth=2, label='Trend')
    axes[1].set_xlabel('Sample', fontsize=12)
    axes[1].set_ylabel('Amplitude', fontsize=10)
    axes[1].set_title('Anomaly (Degradation)', fontsize=12, fontweight='bold')
    axes[1].legend(fontsize=9)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/figures/degradation_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

# %% Cell 7: Rate of Change Analysis
print("\n" + "="*80)
print("RATE OF CHANGE ANALYSIS")
print("="*80)

roc_features = extractor.extract_rate_of_change_features(window, prefix='vib_')

print("\nRate of Change Features:")
for key, value in roc_features.items():
    print(f"  {key}: {value:.4f}")

# Visualize
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Original signal
axes[0].plot(window, linewidth=1, color='blue')
axes[0].set_ylabel('Signal', fontsize=10)
axes[0].set_title('Original Signal', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# First derivative (velocity)
velocity = np.diff(window)
axes[1].plot(velocity, linewidth=1, color='green')
axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
axes[1].set_ylabel('Velocity', fontsize=10)
axes[1].set_title('First Derivative (Rate of Change)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Second derivative (acceleration)
acceleration = np.diff(velocity)
axes[2].plot(acceleration, linewidth=1, color='red')
axes[2].axhline(0, color='black', linestyle='--', linewidth=1)
axes[2].set_xlabel('Sample', fontsize=12)
axes[2].set_ylabel('Acceleration', fontsize=10)
axes[2].set_title('Second Derivative (Acceleration)', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/rate_of_change.png', dpi=300, bbox_inches='tight')
plt.show()

# %% Cell 8: Summary
print("\n" + "="*80)
print("TEMPORAL ANALYSIS SUMMARY")
print("="*80)

# Extract all temporal features
all_temporal = {}
all_temporal.update(trend_features)
all_temporal.update(acf_features)
all_temporal.update(roc_features)
all_temporal.update(extractor.extract_lag_features(window, [10, 50, 100], prefix='vib_'))
all_temporal.update(extractor.extract_cumulative_features(window, prefix='vib_'))
all_temporal.update(deg_normal)

print(f"\n✓ Total temporal features extracted: {len(all_temporal)}")

print(f"\nTemporal Feature Categories:")
print(f"  • Lag features: Capture past values")
print(f"  • Rate of change: Velocity and acceleration")
print(f"  • Trend analysis: Linear and polynomial trends")
print(f"  • Autocorrelation: Detect periodicities and dependencies")
print(f"  • Cross-correlation: Relationships between sensors")
print(f"  • Cumulative statistics: Long-term patterns")
print(f"  • Degradation indicators: Health deterioration")

print(f"\nKey Insights:")
print(f"  • Temporal features capture time-dependent patterns")
print(f"  • Degradation shows increasing trend and variance")
print(f"  • Autocorrelation reveals periodic components")
print(f"  • Cross-correlation shows sensor relationships")
print(f"  • Rate of change detects rapid changes")

print("\n" + "="*80)
print("✓ Temporal analysis complete!")
print("="*80)